### Problem 5.1 - Capacitated transportation problem revisited
*(Marginal costs via dual calculation)*
#' 
Adding necessary packages:

In [1]:
# using Pkg
# Pkg.add("GLPK")

In [2]:
## Packages
using JuMP, Cbc, GLPK # We need GLPK for obtaining duals

We start by defining the problem data

In [3]:
nS = 3 # Number of suppliers
nD = 3 # Number of demand points
nP = 2 # Number of products

S = 1:nS
D = 1:nD
P = 1:nP

A = [] # Set of arcs, we include arcs from all suppliers to all demand points
for s in S, d in D
    push!(A, (s,d))
end

In [4]:
costs = zeros(nP,nS,nD) # Cost of transporting one unit of product d from supplier s to demand point d 
costs[1,:,:] = [5   5   Inf; 
                8   9   7; 
                Inf 10  8]
costs[2,:,:] = [Inf 18 Inf; 
                15  12 14; 
                Inf 20 Inf]

sup = [80  400; 
       200 1500; 
       200 300]
dem = [60  300; 
       100 1000; 
       200 500]

cap = [Inf 300 0; 
       300 700 600; 
       0   Inf Inf];

In [5]:
## Define model using GLPK
model = Model(GLPK.Optimizer);

## Variables
@variable(model, x[a in A, p in P; costs[p,a[1],a[2]] < Inf] >= 0);

## OF
@objective(model, Min, sum(costs[p,a[1],a[2]]*x[a,p] for p in P, a in A if costs[p,a[1],a[2]] < Inf));

## Constraints
@constraint(model, sup_con[s in S, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[1] == s) <= sup[s,p]);       # sum of everything that leaves supplier s
@constraint(model, dem_con[d in D, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[2] == d) >= dem[d,p]);       # sum of everything that arrives at demand d
@constraint(model, cap_con[a in A; cap[a[1],a[2]] < Inf], sum(x[a,p] for p in P if costs[p,a[1],a[2]] < Inf) <= cap[a[1],a[2]]);# arc capacity constraints

In [6]:
set_silent(model) # Actually works with GLPK
optimize!(model)
status = termination_status(model)
println(status)

OPTIMAL


In [7]:
## Saving the optimal value
obj = objective_value(model)

28040.0

Function ``dual()`` in the ``JuMP`` library gives the value of the dual variable associated to the constraint at the optimal solution, in other words the *marginal costs*. Here we need using the elemnt-wise operator ``.`` as we have multiple constraints (check the domains). The *marginal costs* value stands for how much adding one unit more to the constraint's RHS (in the case is a $\leq$ constraint) impacts the final result.<br/>
<br/>
One interpretation for the *marginal costs* in this problem is how much the company is willing to pay for expanding the supplies' or the arcs' capacity (depending on if we're analysing the dual of the supplies or the arcs constraints).

In [8]:
## Computing the duals to infer the marginal costs
mc_supply = dual.(sup_con);
mc_arcs = dual.(cap_con);
for s in S, p in P
    println("The marginal costs for the supply $s for the product $p is: $(mc_supply[s,p])")
end
for a in A
    if cap[a[1],a[2]] < Inf
        println("The marginal costs for the arc $a is: $(mc_arcs[a])")
    end
end
for a in A, p in P
    if costs[p,a[1],a[2]] < Inf
        println("The value of $(x[a,p]) is $(value(x[a,p]))")
    end
end

The marginal costs for the supply 1 for the product 1 is: -3.0
The marginal costs for the supply 1 for the product 2 is: 0.0
The marginal costs for the supply 2 for the product 1 is: 0.0
The marginal costs for the supply 2 for the product 2 is: 0.0
The marginal costs for the supply 3 for the product 1 is: 0.0
The marginal costs for the supply 3 for the product 2 is: 0.0
The marginal costs for the arc (1, 2) is: -2.0
The marginal costs for the arc (1, 3) is: 0.0
The marginal costs for the arc (2, 1) is: 0.0
The marginal costs for the arc (2, 2) is: -8.0
The marginal costs for the arc (2, 3) is: -1.0
The marginal costs for the arc (3, 1) is: 0.0
The value of x[(1, 1),1] is 60.0
The value of x[(1, 2),1] is 20.0
The value of x[(1, 2),2] is 280.0
The value of x[(2, 1),1] is 0.0
The value of x[(2, 1),2] is 300.0
The value of x[(2, 2),1] is 0.0
The value of x[(2, 2),2] is 700.0
The value of x[(2, 3),1] is 100.0
The value of x[(2, 3),2] is 500.0
The value of x[(3, 2),1] is 80.0
The value of x[

In [9]:
## Raising the supply availability of product 1 at the first supply node by 1
sup[1,1] = sup[1,1] + 1;

In [10]:
## Define model using GLPK
model = Model(GLPK.Optimizer);

## Variables
@variable(model, x[a in A, p in P; costs[p,a[1],a[2]] < Inf] >= 0);

## OF
@objective(model, Min, sum(costs[p,a[1],a[2]]*x[a,p] for p in P, a in A if costs[p,a[1],a[2]] < Inf));

## Constraints
@constraint(model, sup_con[s in S, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[1] == s) <= sup[s,p]);       # sum of everything that leaves supplier s
@constraint(model, dem_con[d in D, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[2] == d) >= dem[d,p]);       # sum of everything that arrives at demand d
@constraint(model, cap_con[a in A; cap[a[1],a[2]] < Inf], sum(x[a,p] for p in P if costs[p,a[1],a[2]] < Inf) <= cap[a[1],a[2]]);# arc capacity constraints

In [11]:
set_silent(model)
optimize!(model)
status = termination_status(model)
println(status)

OPTIMAL


In [12]:
## New optimal value
new_obj = objective_value(model)

28037.0

In [13]:
## Decrease in the optimal value
new_obj - obj

-3.0

In [14]:
mc_supply[1,1]

-3.0

The marginal cost predicted the change in objective value correctly. Let's now try changing another bound.

In [15]:
## Back to the original supply availability
sup[1,1] = sup[1,1] - 1;

In [16]:
sup[1,1]

80

In [17]:
## Increasing the arc capacity by 1 for the arc from supplier 2 to demand node 2
cap[2, 2] = cap[2, 2] + 1;

In [18]:
## Define model using GLPK
model = Model(GLPK.Optimizer);

## Variables
@variable(model, x[a in A, p in P; costs[p,a[1],a[2]] < Inf] >= 0);

## OF
@objective(model, Min, sum(costs[p,a[1],a[2]]*x[a,p] for p in P, a in A if costs[p,a[1],a[2]] < Inf));

## Constraints
@constraint(model, sup_con[s in S, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[1] == s) <= sup[s,p]);       # sum of everything that leaves supplier s
@constraint(model, dem_con[d in D, p in P], sum(x[a,p] for a in A if costs[p,a[1],a[2]] < Inf && a[2] == d) >= dem[d,p]);       # sum of everything that arrives at demand d
@constraint(model, cap_con[a in A; cap[a[1],a[2]] < Inf], sum(x[a,p] for p in P if costs[p,a[1],a[2]] < Inf) <= cap[a[1],a[2]]);# arc capacity constraints

In [19]:
set_silent(model)
optimize!(model)
status = termination_status(model)
println(status)

OPTIMAL


In [20]:
## New optimal value
new_obj = objective_value(model)

28039.0

In [21]:
new_obj-obj

-1.0

In [22]:
mc_arcs[(2, 2)]

-8.0

Turns out the marginal cost that we calculated did not predict this change correctly. Comparing the solution below, we notice that one unit of product 1 to demand point 2 is now transported from supplier 2 instead of supplier 3 due to the increased capacity of arc 2->2. This shows that you should always check whether you can apply marginal costs. **If the optimal basis changes because of a change in $b$, you can't apply marginal costs.**

In [23]:
## Computing the duals to infer the marginal costs
mc_supply = dual.(sup_con);
mc_arcs = dual.(cap_con);
for s in S, p in P
    println("The marginal costs for the supply $s for the product $p is: $(mc_supply[s,p])")
end
for a in A
    if cap[a[1],a[2]] < Inf
        println("The marginal costs for the arc $a is: $(mc_arcs[a])")
    end
end
for a in A, p in P
    if costs[p,a[1],a[2]] < Inf
        println("The value of $(x[a,p]) is $(value(x[a,p]))")
    end
end

The marginal costs for the supply 1 for the product 1 is: -3.0
The marginal costs for the supply 1 for the product 2 is: 0.0
The marginal costs for the supply 2 for the product 1 is: 0.0
The marginal costs for the supply 2 for the product 2 is: -7.0
The marginal costs for the supply 3 for the product 1 is: 0.0
The marginal costs for the supply 3 for the product 2 is: 0.0
The marginal costs for the arc (1, 2) is: -2.0
The marginal costs for the arc (1, 3) is: 0.0
The marginal costs for the arc (2, 1) is: 0.0
The marginal costs for the arc (2, 2) is: -1.0
The marginal costs for the arc (2, 3) is: -1.0
The marginal costs for the arc (3, 1) is: 0.0
The value of x[(1, 1),1] is 60.0
The value of x[(1, 2),1] is 20.0
The value of x[(1, 2),2] is 280.0
The value of x[(2, 1),1] is 0.0
The value of x[(2, 1),2] is 300.0
The value of x[(2, 2),1] is 1.0
The value of x[(2, 2),2] is 700.0
The value of x[(2, 3),1] is 100.0
The value of x[(2, 3),2] is 500.0
The value of x[(3, 2),1] is 79.0
The value of x

### Problem 5.5 - Complementary slackness
Recall the paint factory problem introduced in Section 1.2.1. of the lecture notes

In [24]:
A = [ 6  4; 
      1  2; 
     -1  1; 
      0  1]
b = [24; 
     6; 
     1; 
     2]
c = [5; 
     4];

#### Solving the primal and dual problems
Formulate and solve the two problems to obtain their optimal solutions

*Primal Problem Formulation*

\begin{align*}
\max_x \ & z = 5x_1 + 4x_2 \\
\text{s.t.: } & 6x_1 + 4x_2 \leq 24 & (p_1)\\
& x_1 + 2x_2 \leq 6 & (p_2)\\
& x_2 - x_1 \leq 1 & (p_3)\\
& x_2 \leq 2 & (p_4)\\
& x_1, x_2 \geq 0
\end{align*}

*Succinct Form*
\begin{align*}
\max_x \ & z = \mathbf{c}^\top \mathbf{x} \\
\text{s.t.: } & \mathbf{A} \mathbf{x} \leq \mathbf{b} & (\mathbf{p})\\
& \mathbf{x} \geq \mathbf{0}
\end{align*}

In [38]:
# TODO: add your code here
model_p = Model(GLPK.Optimizer)

@variable(model_p, x[1:2] >= 0)
@objective(model_p, Max, c'*x)
@constraint(model_p, primal_constraints, A*x .<= b)

set_silent(model_p)
optimize!(model_p)

status = termination_status(model_p)
println(status)

@show objective_value(model_p);


OPTIMAL
objective_value(model_p) = 21.0


*Dual Problem Formulation*

\begin{align*}
\min_x \ & z = 24p_1 + 6p_2 + p_3 + 2p_4 \\
\text{s.t.: } & 6p_1 + p_2 - p_3 \geq 5 & (x_1)\\
& 4p_1 + 2p_2 + p_3 + p_4 \geq 4 & (x_2)\\
& p_1, p_2, p_3, p_4 \geq 0
\end{align*}

*Succinct Form*
\begin{align*}
\min_p \ & z = \mathbf{p}^\top \mathbf{b} \\
\text{s.t.: } & \mathbf{A}^\top \mathbf{p} \geq \mathbf{c} & (\mathbf{x})\\
& \mathbf{p} \geq \mathbf{0}
\end{align*}

In [42]:
# TODO: add your code here
model_d = Model(GLPK.Optimizer)

@variable(model_d, p[1:4] >= 0)
@objective(model_d, Min, p'*b)
@constraint(model_d, dual_constraints, A'*p .>= c)

set_silent(model_d)
optimize!(model_d)

status = termination_status(model_d)
println(status)

@show objective_value(model_d);

OPTIMAL
objective_value(model_d) = 21.0


#### Complementary slackness
Verify the optimality of your solutions using complementary slackness

In [45]:
# TODO: add your code here

x_opt = value.(model_p[:x])
p_opt = value.(model_d[:p])

VarPrimalConstraints = dual.([con for con in primal_constraints])
VarDualConstraints = dual.([con for con in dual_constraints])

@show x_opt, VarDualConstraints
@show p_opt, VarPrimalConstraints

lhs_primal = A*x_opt
lhs_dual = p_opt'*A

println("Primal feasibility:")
for m in 1:4
    println("$(round(lhs_primal[m],digits=2)) <= $(b[m])")
end

println("\nDual feasibility:")
for n in 1:2
    println("$(round(lhs_dual[n],digits=2)) >= $(c[n])")
end

println("\nComplementary slackness:")
for m in 1:4
    println("$(round((lhs_primal[m] - b[m])*p_opt[m],digits=2)) = 0")
end
for n in 1:2
    println("$(round((lhs_dual[n] - c[n])*x_opt[n],digits=2)) = 0")
end

# Primal optimal: p^T (Ax-b) = 0

# Dual optimal: (c^T - p^T A)x = 0


(x_opt, VarDualConstraints) = ([3.0, 1.4999999999999998], [3.0, 1.4999999999999998])
(p_opt, VarPrimalConstraints) = ([0.75, 0.5000000000000001, 0.0, 0.0], [-0.7500000000000001, -0.49999999999999983, -0.0, -0.0])
Primal feasibility:
24.0 <= 24
6.0 <= 6
-1.5 <= 1
1.5 <= 2

Dual feasibility:
5.0 >= 5
4.0 >= 4

Complementary slackness:
0.0 = 0
0.0 = 0
-0.0 = 0
-0.0 = 0
0.0 = 0
0.0 = 0
